In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

## Install mmaction

In [3]:
!pip install -U openmim
!mim install mmengine
!mim install mmcv
!mim install mmdet
!mim install mmpose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 24.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━

In [4]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.13.0+cu117 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -e .
!pip install -r requirements/optional.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 836.5 kB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torchvision==0.6.1+cu101 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.12.0, 0.12.0+cpu, 0.12.0+cu102, 0.12.0+cu113, 0.12.0+cu115, 0.12.0+rocm4.3.1, 0.12.0+rocm4.5.2, 0.13.0, 0.13.0+cpu, 0.13.0+cu102, 0.13.0+cu113, 0.13.0+cu116, 0.13.0+rocm5.0, 0.13.0+rocm5.1.1, 0.13.1, 0.13.1+cpu, 0.13.1+cu102, 0.13.1+cu113, 0.13.1+cu116, 0.13.1+rocm5.0, 0.13.1+rocm5.1.1, 0.14.0, 0.14.0+cpu, 0.14.0+cu116, 0.14.0+cu117, 0.14.0+rocm5.1.1, 0.14.0+rocm5.2, 0.14.1, 0.14.1+cpu, 0.14.1+cu116, 0.14.1+cu117, 0.14.1+rocm5.1.1, 0.14.1+rocm5.2, 0.15.0, 0.15.0+cpu, 0.15.0+cu117, 0.15.0+cu118, 0.15.0+rocm5.3, 0.15.0+rocm5.4.2, 0.15.1, 0.15.1+cpu, 0.15.1+cu117, 0.1

## Download dataset for inference

In [5]:
%cd /content/mmaction2/tools/data/ucf101

/content/mmaction2/tools/data/ucf101


In [6]:
!bash download_annotations.sh
!bash download_videos.sh
clear_output()

In [7]:
!bash generate_rawframes_filelist.sh
!bash generate_videos_filelist.sh

parse frames under directory data/ucf101/rawframes/
frame directory analysis done
Filelist for rawframes generated.
Filelist for videos generated.


In [8]:
%cd /content/mmaction2

/content/mmaction2


## Take samples for inference

In [9]:
# Generate Label lists

with open('/content/mmaction2/tools/data/ucf101/label_map.txt') as f:
    lines = f.readlines()

labels = [l.split('\n')[0] for l in lines]

print('No. of labels :',len(labels))

No. of labels : 101


In [10]:
# The validation split 3 will be used for inference

with open('/content/mmaction2/data/ucf101/ucf101_val_split_3_videos.txt') as f:
    lines = f.readlines()

valsplit_3 = [l.split('\n')[0] for l in lines]

In [11]:
from random import sample
import os

samples = sample(valsplit_3,5)  # Take 5 random sample

In [12]:
samples

['HeadMassage/v_HeadMassage_g18_c03.avi 38',
 'PoleVault/v_PoleVault_g18_c06.avi 67',
 'WallPushups/v_WallPushups_g18_c05.avi 98',
 'WritingOnBoard/v_WritingOnBoard_g17_c04.avi 99',
 'PlayingPiano/v_PlayingPiano_g21_c04.avi 63']

In [14]:
samples_list = []
for sam in samples:
  vid = '/content/mmaction2/data/ucf101/videos/'+ sam.split(' ')[0]
  lab = labels[int(sam.split(' ')[1])]
  samples_list.append([vid,lab])

In [15]:
!cp -r '/content/drive/MyDrive/work_dirs/tsn_r50_1x1x8_50e_ucf101_kinetics400_rgb' .  # copy the checkpoint file

In [16]:
%cd /content/mmaction2/

/content/mmaction2


## Create Model from checkpoint file

In [17]:
from mmaction.apis import inference_recognizer, init_recognizer

config_path = '/content/mmaction2/tsn_r50_1x1x8_50e_ucf101_kinetics400_rgb/tsn.py'
checkpoint_path = '/content/mmaction2/tsn_r50_1x1x8_50e_ucf101_kinetics400_rgb/best_acc_top1_epoch_15.pth' # can be a local path

# build the model from a config file and a checkpoint file
model = init_recognizer(config_path, checkpoint_path, device="cpu")  # device can be 'cuda:0'


Loads checkpoint by local backend from path: /content/mmaction2/tsn_r50_1x1x8_50e_ucf101_kinetics400_rgb/best_acc_top1_epoch_15.pth


## Inference

In [18]:
%cd /content/

/content


In [19]:
import cv2
from random import randrange
import os
from tqdm import tqdm
from glob import glob
def convert_frames2video(img_pth,vid_name):  # Detected frames into video
  img_array = []
  for filename in tqdm(sorted(glob(img_pth+'/*.jpg'))):
      img = cv2.imread(filename)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
      # print(filename, ' - ', size)
  out = cv2.VideoWriter(vid_name+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
  
  for i in range(len(img_array)):
      out.write(cv2.cvtColor(img_array[i], cv2.COLOR_RGB2BGR))
  out.release()
  print('Converting completed. '+vid_name+'.avi created ...') 
  
def inference_video(vid_path,label_lt):                    # Predict the action name and score
  if not os.path.isfile(vid_path) : print('Incorrect video path.')
  result = inference_recognizer(model,vid_path)
  lb = result.pred_labels.item[0]
  sc = result.pred_scores.item[lb]

  return label_lt[lb],sc

def write_inference_frame(dest_dir,vid_path,score,label,font_size=1,thickness=1):  # Write the predicted frame 
  if not os.path.isdir(dest_dir): os.mkdir(dest_dir)
  video = cv2.VideoCapture(vid_path)
  frame_width = int(video.get(3))
  frame_height = int(video.get(4))
  coor = (int(frame_width/15),int(frame_height-(frame_height/10)))
  label_txt = '[{:.2f}] - {}'.format(score,label)
  # rand_color = (randrange(255),randrange(255),randrange(255))

  # fps = video.get(cv2.CAP_PROP_FPS)
  # print('frames per second =',fps)
  count = 1
  while(True):
        
      # Capture frames in the video
      ret, frame = video.read()
    
      # describe the type of font
      # to be used.
      font = cv2.FONT_HERSHEY_SIMPLEX
    
      # Use putText() method for
      # inserting text on video
      cv2.putText(frame, 
                  label_txt, 
                  coor, 
                  font, font_size, 
                  (202, 231, 20), 
                  thickness, 
                  cv2.LINE_4)
      frame_name = dest_dir+'/'+str(count).zfill(4)+'.jpg'
      try :
        cv2.imwrite(frame_name,frame)
        count += 1
      except:
        break;
  return frame_width,frame_height

In [20]:
l,s = inference_video(vd1,labels)  

NameError: ignored

In [ ]:
dest = '/content/test'


In [21]:
## Inference ##

for i,a in enumerate(samples_list):
  dest_pth = '/content/testvid_'+str(i+1) # To save the predicted frames
  vid,lab = a
  l,s = inference_video(vid,labels)  
  write_inference_frame(dest_pth,vid,s,l,0.5)
  print(vid.split('/')[-1],',   predict-',l,',   score-',s)
  if not os.path.isdir('/content/inference'):os.mkdir('/content/inference') # To save the video after prediction each frames
  
  convert_frames2video(dest_pth, f'/content/inference/testvid_{str(i+1)}')

05/22 05:00:32 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
05/22 05:00:32 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
v_HeadMassage_g18_c03.avi ,   predict- HeadMassage ,   score- tensor(0.9952)


100%|██████████| 172/172 [00:00<00:00, 518.79it/s]


Converting completed. /content/inference/testvid_1.avi created ...
v_PoleVault_g18_c06.avi ,   predict- PoleVault ,   score- tensor(0.9996)


100%|██████████| 143/143 [00:00<00:00, 932.51it/s]


Converting completed. /content/inference/testvid_2.avi created ...
v_WallPushups_g18_c05.avi ,   predict- WallPushups ,   score- tensor(1.0000)


...

Converting completed. /content/inference/testvid_3.avi created ...
v_WritingOnBoard_g17_c04.avi ,   predict- WritingOnBoard ,   score- tensor(0.9995)


100%|██████████| 206/206 [00:00<00:00, 1055.56it/s]


Converting completed. /content/inference/testvid_4.avi created ...
v_PlayingPiano_g21_c04.avi ,   predict- PlayingPiano ,   score- tensor(1.)


100%|██████████| 217/217 [00:00<00:00, 598.23it/s]


Converting completed. /content/inference/testvid_5.avi created ...


In [23]:
clip=VideoFileClip('/content/inference/testvid_1.avi')
clip.ipython_display(width=280)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


## Concatenate the videos

In [24]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [25]:
final_clip = []
for vids in os.listdir('/content/inference'):
  testvid = os.path.join('/content/inference',vids)
  clip = VideoFileClip(testvid)
  final_clip.append(clip)

In [26]:
final_clip = concatenate_videoclips(final_clip)
final_clip.write_videofile("new.mp4") #16 seconds video

Moviepy - Building video new.mp4.
Moviepy - Writing video new.mp4



Moviepy - Done !
Moviepy - video ready new.mp4


In [27]:
clip=VideoFileClip('/content/new.mp4')
clip.ipython_display(width=280)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
